<a href="https://colab.research.google.com/github/Dianna22/NLP/blob/master/Project/Emocontext/elmo_turns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q emoji

    100% |████████████████████████████████| 51kB 3.8MB/s 


In [0]:
# import tensorflow as tf
# tf.test.gpu_device_name()

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat /content/gdrive/My\ Drive/foo.txt

In [1]:
import csv
import emoji
from keras import backend as K
from keras import optimizers
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.engine import Layer
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Add, Average, Maximum
from keras.layers import Dropout, Input, TimeDistributed, PReLU
from keras.models import Sequential, Model, load_model, save_model, model_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.utils import to_categorical
from nltk.tokenize import TweetTokenizer
import numpy as np
import pandas as pd
import re
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import tensorflow as tf
import tensorflow_hub as hub
import time

Using TensorFlow backend.
W0410 05:13:44.555818 140604992288640 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


#### Elmo layer

In [0]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

#     def compute_mask(self, inputs, mask=None):
#         return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

#### Path vars & load

In [0]:
path_prefix='/content/gdrive/My Drive/NLP/'

In [0]:
# from google.colab import files
# uploaded = files.upload()

In [0]:
TRAIN_FILE = path_prefix + 'train.txt'
DEV_FILE =  path_prefix + 'dev.txt'
TEST_FILE =  path_prefix + 'test.txt'

TURNS_NAMES = ["turn1", "turn2", "turn3"]
LABEL = ["label"]
CONCATENATED_TURNS = "turns"
def parse_file(file_path):
    output_dict = dict()
    with open(file_path, newline='\n', encoding='utf8') as csvfile:
        return pd.read_csv(csvfile, sep="\t")

In [0]:
MODEL_CHECKPOINT = path_prefix + '3_elmo-turns'

In [0]:
train_data = parse_file(TRAIN_FILE)
dev_data = parse_file(DEV_FILE)
test_data = parse_file(TEST_FILE)

In [0]:
print(np.shape(train_data))

(30160, 5)


In [0]:
def concatenate_turns(df, delim="fullstop"):
    turns = [("%s %s %s %s %s" %
                 (row[TURNS_NAMES[0]], delim,
                  row[TURNS_NAMES[1]], delim,
                  row[TURNS_NAMES[2]])).lower()
                 for index, row in df.iterrows()]
    df[CONCATENATED_TURNS] = pd.Series(turns, index=df.index)
    return df

In [0]:
def emoticons_replace(df):
    for index, row in df.iterrows():
        for turn in range(3):
          turns = emoji.demojize(row[TURNS_NAMES[turn]])
          # remove delimiters ":"  (:smiley: -> smiley)
          for emoj in re.findall(":\w*:", turns):
              turns  = turns.replace(emoj, emoj[1:-1])
          df.at[index, TURNS_NAMES[turn]] = turns
    return df

In [0]:
train = emoticons_replace(train_data)
dev = emoticons_replace(dev_data)
test = emoticons_replace(test_data)

In [0]:
max_sentence = [163,82,189]
### angry: [1 0 0 0]
### happy: [0 1 0 0]
### others: [0 0 1 0]
### sad: [0 0 0 1]
labels = {0: 'angry',
          1: 'happy',
          2: 'others',
          3: 'sad'}

In [0]:
### angry: [1 0 0 0]
### happy: [0 1 0 0]
### others: [0 0 1 0]
### sad: [0 0 0 1]
Y_train = pd.get_dummies(train[LABEL]).as_matrix()
# for i, t in enumerate(train[LABEL].iterrows()):
#     if t[1]['label']=='others':
#         print(Y_train[i])
#         break
Y_dev = pd.get_dummies(dev[LABEL]).as_matrix()
Y_test = pd.get_dummies(test[LABEL]).as_matrix()

#### Model Callback

In [0]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict([self.validation_data[:3][i] for i in range(3)]))).round()
        val_targ = self.validation_data[3]

        _val_f1 = f1_score(val_targ, val_predict, average='micro')
        _val_recall = recall_score(val_targ, val_predict, average='micro')
        _val_precision = precision_score(val_targ, val_predict, average='micro')

        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print("— val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
        f1 = print_metrics_predicted(val_predict, val_targ, MODEL_CHECKPOINT+"-results")
        if f1 >max_f1:
          self.model.save(MODEL_CHECKPOINT)
        return
 
metrics = Metrics()

def print_metrics_predicted(predicts,Y,filename):
    tp =[0,0,0,0]
    fp =[0,0,0,0]
    fn =[0,0,0,0]
    for i,pred in enumerate(predicts):
        p = np.argmax(pred)
        y = np.argmax(Y[i])
        if p == y:
            tp[p] += 1
        else:
            fp[p] +=1
            fn[y] +=1
    prec = sum(tp)/(sum(tp+fp)+np.finfo(float).eps)
    rec = sum(tp)/(sum(tp+fn)+np.finfo(float).eps)
    with open(filename, "w") as f:
      print("F1 all")
      f1_all = 2*prec*rec/(prec+rec+np.finfo(float).eps)
      print(f1_all) 
      f.write(str(f1_all))
      print("***")
      for i in range(4):
          print("F1 %s: " % labels[i])
          f.write("\n%s: " % labels[i])
          prec = tp[i]/(tp[i]+fp[i]+np.finfo(float).eps)
          rec = tp[i]/(tp[i]+fn[i]+np.finfo(float).eps)
          f1 = 2*prec*rec/(prec+rec+np.finfo(float).eps)
          print(f1)
          f.write(str(f1))
          print("****")
      tp.pop(2)
      fp.pop(2)
      fn.pop(2)
      print("F1 happy angry sad")
      f.write("\nF1 happy angry sad: ")
      prec = sum(tp)/(sum(tp+fp)+np.finfo(float).eps)
      rec = sum(tp)/(sum(tp+fn)+np.finfo(float).eps)
      f1= 2*prec*rec/(prec+rec+np.finfo(float).eps)
      f.write(str(f1))
      print(f1)
      return f1


##### Model params


In [0]:
# vocabulary_size = len(tokenizer.word_counts) + 1

epochs = 10
embed_dim = 256
lstm_out = 128
batch_size = 128
drop_out = 0.3
loss_fct = 'binary_crossentropy'
activation_fct = 'softmax'
optimizer = "Adam-0.01"

parameters = """Epochs:%s\nEmbed_dim: %s\nLstm_out: %s\nBatch size: %s\nDrop_out: %s
Loss_fct: %s\nActivaion_fct: %s\nOptimizer: %s\n
""" %(str(epochs), str(embed_dim), str(lstm_out), str(batch_size), str(drop_out), loss_fct,
      activation_fct, optimizer)

###### Modele esuate

In [0]:
# input_layer = Input(shape=(max_sentence,), dtype='int32')

# embedding_layer = Embedding(vocabulary_size,
#                             embed_dim,
#                             weights=[embedding_matrix],
#                             input_length=max_sentence,
#                             trainable=False)(input_layer)
# bi_lstm = Bidirectional(LSTM(lstm_out))(embedding_layer)
# dropout = Dropout(0.4)(bi_lstm)
# dense = Dense(128,activation=activation_fct)(dropout)
# dropout = Dropout(0.2)(dense)
# dense = Dense(4,activation=activation_fct)(dropout)
# adam = optimizers.Adam(lr=0.01)
# rmsprop = optimizers.RMSprop(lr=0.005)#, rho=0.9, epsilon=None, decay=0.0)
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# model = Model(inputs=[input_layer], outputs=dense)
# model.compile(loss = loss_fct, optimizer=adam, metrics = ['accuracy'])
# print(model.summary())

In [0]:
# input_layer = Input(shape=(1,), dtype="string")
# embedding = ElmoEmbeddingLayer()(input_layer)
# # bilstm = Bidirectional(LSTM(lstm_out), input_shape=(1024,))(embedding)dr = Dropout(0.4)(embedding)
# dropout = Dropout(0.4)(embedding)
# dense = Dense(128,activation=activation_fct)(dropout)
# dropout = Dropout(0.2)(dense)
# dense = Dense(4,activation=activation_fct)(dropout)

# adam = optimizers.Adam(lr=0.01)
# rmsprop = optimizers.RMSprop(lr=0.005)#, rho=0.9, epsilon=None, decay=0.0)
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# model = Model(inputs=[input_layer], outputs=dense)
# model.compile(loss = 'binary_crossentropy', optimizer=adam, metrics = ['accuracy'])
# print(model.summary())

In [0]:
# input_text = Input(shape=(1,), dtype="string")
# embedding = ElmoEmbeddingLayer()(input_text)
# # bilstm = Bidirectional(LSTM(lstm_out), input_shape=(1024,))(embedding)dr = Dropout(0.4)(embedding)
# dense = Dense(1024, activation='relu')(embedding)
# dense = Dense(512, activation='relu')(dense)
# dense = Dropout(0.15)(dense)
# dense = Dense(128, activation='relu')(dense)
# dense = Dropout(0.1)(dense)
# pred = Dense(4, activation='softmax')(dense)

# model = Model(inputs=[input_text], outputs=pred)
# adam = optimizers.Adam(lr=0.003)
# model.compile(loss='mean_squared_error', optimizer= adam, metrics=['accuracy'])
# model.summary()

##### Model

In [116]:
input_layer1 = Input(shape=(1,), dtype="string")
embedding1 = ElmoEmbeddingLayer()(input_layer1)
input_layer2 = Input(shape=(1,), dtype="string")
embedding2 = ElmoEmbeddingLayer()(input_layer2)
input_layer3 = Input(shape=(1,), dtype="string")
embedding3 = ElmoEmbeddingLayer()(input_layer3)

merged = Add()([embedding1, embedding2, embedding3])

dropout = Dropout(0.4)(merged)
dense = Dense(128,activation=activation_fct)(dropout)
dropout = Dropout(0.2)(dense)
dense = Dense(4,activation=activation_fct)(dropout)

adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.005)#, rho=0.9, epsilon=None, decay=0.0)
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model = Model(inputs=[input_layer1, input_layer2, input_layer3], outputs=dense)
model.compile(loss = 'binary_crossentropy', optimizer=adam, metrics = ['accuracy'])
print(model.summary())

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0410 06:41:33.276705 140604992288640 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0410 06:41:37.138902 140604992288640 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0410 06:41:41.171009 140604992288640 saver.py:1483] Saver not created because there are no variables in the graph to restore


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
elmo_embedding_layer_19 (ElmoEm (None, 1024)         4           input_19[0][0]                   
__________________________________________________________________________________________________
elmo_embed

In [0]:
max_f1 = 0
model.fit([train[TURNS_NAMES[i]] for i in range(3)], Y_train, epochs=100, verbose=1,
          batch_size=128,
          validation_data=([dev[TURNS_NAMES[i]] for i in range(3)], Y_dev),
          callbacks=[metrics,
                     EarlyStopping(patience=5),
                     ModelCheckpoint(MODEL_CHECKPOINT, monitor='val_loss', verbose=1, save_best_only=True, mode='max')
                    ])
#0.32-0.34
#0.26-0.37
#-droppout
#0.37-0.40
#+prelu
#0.36-0.37
#-prelu
# 0.33-0.46
# mse
# 0.44 - loss 0.08 val loss 0.08
# adam 0.03
with open(MODEL_CHECKPOINT + ".json","w") as f:
  f.write(model.to_json())

Train on 30160 samples, validate on 2755 samples
Epoch 1/100
17792/30160 [================>.............] - ETA: 2:19 - loss: 0.5172 - acc: 0.7480

In [0]:
with open(MODEL_CHECKPOINT + ".json", "r") as f:
  model = model_from_json(f.read(), custom_objects={'ElmoEmbeddingLayer':ElmoEmbeddingLayer()})
  
model.load_weights(MODEL_CHECKPOINT)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0405 05:43:45.993275 140035057215360 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
# import os
# print(os.listdir(os.getcwd()))

In [0]:
# from google.colab import files
# files.download('3_elmo-dense-prelu')

In [0]:
# loaded_model = load_model(MODEL_CHECKPOINT, custom_objects={'ElmoEmbeddingLayer':ElmoEmbeddingLayer()})

In [0]:
predicts = (np.asarray(model.predict(test[CONCATENATED_TURNS][:3]))).round()

In [0]:
print(predicts[:10])

In [0]:
print_metrics_predicted(predicts, Y_test, filename=MODEL_CHECKPOINT+"-results")

F1 all
0.6666666666666665
***
F1 angry: 
0.0
****
F1 happy: 
0.0
****
F1 others: 
0.7999999999999999
****
F1 sad: 
0.0
****
F1 happy angry sad
0.0


0.0

In [0]:
print("running")
predicts =
f1 = print_metrics(model, test[CONCATENATED_TURNS], Y_test, "3_Elmo-2epochs-results.txt")
print(f1)

In [0]:
print(model)